<a href="https://colab.research.google.com/github/aivrm/ai/blob/main/EX_6_7_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%A9%8B%EC%A7%84_%EC%9E%91%EC%82%AC%EA%B0%80_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
## 6-7. 프로젝트: 멋진 작사가 만들기

---

In [30]:
import tensorflow as tf

print(tf.__version__)

2.8.2


---
### Step 1. 데이터 다운로드

---

---
### Step 2. 데이터 읽어오기

---

In [31]:
import glob
import os, re

BASE_DIR = '/content/drive/MyDrive/_aiffel'
# txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_file_path = BASE_DIR+'/aiffel/lyricist/data/lyrics/*'
txt_files_path_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_files_path_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


---
### Step 3. 데이터 정제

---

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [32]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


In [33]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [34]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if len(sentence) > 15: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> baby , baby <end>',
 '<start> ooh , ooh <end>',
 '<start> baby , baby <end>',
 '<start> ooh , ooh <end>',
 '<start> baby , baby <end>',
 '<start> i miss my baby <end>',
 '<start> i miss my baby <end>',
 '<start> you played it <end>',
 '<start> you played it <end>',
 '<start> oh , anymore <end>']

---
### Step 4. 평가 데이터셋 분리

---

tokenize() 함수로 데이터를 Tensor로 변환한 후,<br>
sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다.<br>
단어장의 크기는 12,000 이상 으로 설정하세요!<br>
총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [35]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    print(tokenizer)
    print(type(tokenizer))
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

<class 'keras_preprocessing.text.Tokenizer'>
[[   2   24    5 ...    0    0    0]
 [   2   44    5 ...    0    0    0]
 [   2   24    5 ...    0    0    0]
 ...
 [   2 1074 1075 ...    0    0    0]
 [   2 1074 1075 ...    0    0    0]
 [   2 1074 1075 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f3526041dd0>


In [36]:
print(tensor[:3, :10])

[[ 2 24  5 24  3  0  0  0  0  0]
 [ 2 44  5 44  3  0  0  0  0  0]
 [ 2 24  5 24  3  0  0  0  0  0]]


In [37]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : .
7 : you
8 : oh
9 : it
10 : me


In [38]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[ 2 24  5 24  3  0  0  0  0  0  0  0]
[24  5 24  3  0  0  0  0  0  0  0  0]


In [39]:
# <코드 작성>
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

In [40]:
enc_train.shape, enc_val.shape, dec_train.shape, dec_val.shape

((11296, 12), (2824, 12), (11296, 12), (2824, 12))

In [41]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
ds_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
ds_train = ds_train.shuffle(BUFFER_SIZE)
ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
ds_train

<BatchDataset element_spec=(TensorSpec(shape=(256, 12), dtype=tf.int32, name=None), TensorSpec(shape=(256, 12), dtype=tf.int32, name=None))>

In [42]:
ds_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
ds_val = ds_val.shuffle(BUFFER_SIZE)
ds_val = ds_val.batch(BATCH_SIZE, drop_remainder=True)
ds_val

<BatchDataset element_spec=(TensorSpec(shape=(256, 12), dtype=tf.int32, name=None), TensorSpec(shape=(256, 12), dtype=tf.int32, name=None))>

---
### Step 5. 인공지능 만들기

---

In [43]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [44]:
# embedding_size = 256
# hidden_size = 1024
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [45]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in ds_train.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
lyricist(src_sample)

<tf.Tensor: shape=(256, 12, 12001), dtype=float32, numpy=
array([[[-4.61391282e-05,  6.27216214e-05,  9.78134267e-05, ...,
         -1.37110677e-04,  1.01833626e-04,  1.20703808e-04],
        [-1.38541567e-04, -5.59218788e-05,  3.35400546e-05, ...,
         -1.32436951e-04,  2.09406615e-04,  2.24464631e-04],
        [-1.50381122e-04, -1.90594626e-04,  8.48600976e-05, ...,
         -1.57226663e-04,  4.37522016e-04,  4.09215456e-04],
        ...,
        [-1.17777367e-04, -7.92014820e-04,  9.82160680e-04, ...,
          8.95825622e-04,  1.56365579e-03,  2.18114676e-03],
        [-1.75750771e-04, -1.04601693e-03,  1.10965827e-03, ...,
          1.19454728e-03,  1.62180117e-03,  2.29245424e-03],
        [-2.42254304e-04, -1.33179291e-03,  1.23236212e-03, ...,
          1.47638330e-03,  1.65655173e-03,  2.34865956e-03]],

       [[-4.61391282e-05,  6.27216214e-05,  9.78134267e-05, ...,
         -1.37110677e-04,  1.01833626e-04,  1.20703808e-04],
        [ 8.48829877e-05,  4.30717264e-05, -1

In [46]:
lyricist.summary()

Model: "text_generator_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  3072256   
                                                                 
 lstm_2 (LSTM)               multiple                  5246976   
                                                                 
 lstm_3 (LSTM)               multiple                  8392704   
                                                                 
 dense_1 (Dense)             multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [47]:
# optimizer와 loss등은 차차 배웁니다 
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요

# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

optimizer = tf.keras.optimizers.Adam()

Loss는 아래 제시된 Loss 함수를 그대로 사용하세요!

In [48]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [49]:
# compile
lyricist.compile(loss=loss, optimizer=optimizer)

# model.fit() 함수에 들어가는 다양한 인자를 알고 싶다면 아래의 문서를 참고하세요. 
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit

#lyricist.fit(dataset, epochs=10)
lyricist.fit(ds_train,
             batch_size=256,
             epochs=10,
             validation_data=ds_val,
             verbose=1)

Epoch 1/10
44/44 [==============================] - 6s 100ms/step - loss: 3.0142 - val_loss: 1.9794
Epoch 2/10
44/44 [==============================] - 4s 86ms/step - loss: 1.7371 - val_loss: 1.6485
Epoch 3/10
44/44 [==============================] - 4s 86ms/step - loss: 1.4881 - val_loss: 1.5367
Epoch 4/10
44/44 [==============================] - 4s 88ms/step - loss: 1.4130 - val_loss: 1.4906
Epoch 5/10
44/44 [==============================] - 4s 86ms/step - loss: 1.3769 - val_loss: 1.4747
Epoch 6/10
44/44 [==============================] - 4s 86ms/step - loss: 1.3554 - val_loss: 1.4623
Epoch 7/10
44/44 [==============================] - 4s 86ms/step - loss: 1.3368 - val_loss: 1.4494
Epoch 8/10
44/44 [==============================] - 4s 86ms/step - loss: 1.3169 - val_loss: 1.4362
Epoch 9/10
44/44 [==============================] - 4s 86ms/step - loss: 1.2937 - val_loss: 1.4203
Epoch 10/10
44/44 [==============================] - 4s 86ms/step - loss: 1.2679 - val_loss: 1.4004


In [50]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [51]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [52]:
#generate_text(model, tokenizer, init_sentence="<start> he")
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it <end> '

In [55]:
generate_text(lyricist, tokenizer, init_sentence="<start> i miss", max_len=20)

'<start> i miss it <end> '

---
## 회고

---

이번 노드는, 초반에는 재미있었는데, 후반에서는 개인적으로 힘들고 혼란스러웠습니다.<br>
RNN을 처음으로 접한 기회였지만, 기대했던 결과가 나오지 않아서 아쉽고,<br>
무엇이 문제였는지 좀 더 공부를 하여야 하겠습니다.
